## Setup: Copy Scripts

In [ ]:
!cp /kaggle/input/fds-mental-health-scripts/*.py /kaggle/working/

## Step 1: Train Stage 1 Model (Behavioral Forecasting)

This model:
- Trains on StudentLife sensor data (real correlations)
- Predicts NEXT DAY's behavior (sleep, activity, screen time, social)
- Outputs mean + uncertainty for each prediction

In [ ]:
!python train_stage1_behavioral_kaggle.py

## Step 2: Run Two-Stage Pipeline

This script:
1. Loads Stage 1 model (behavioral forecasting)
2. Loads Stage 2 model (mental health prediction from synthetic data)
3. For each student:
   - Stage 1: Predicts day 8 behavior from days 1-7
   - Stage 2: Uses predicted behavior to infer mental health
4. Tracks uncertainty propagation

In [ ]:
# First, need to upload Stage 2 model as Kaggle dataset
# For now, we'll use the existing mental_health_lstm.pt

!python run_two_stage_pipeline_kaggle.py

## Step 3: Analyze Results

In [ ]:
import json
import pandas as pd
import numpy as np

# Load results
with open('/kaggle/working/two_stage_predictions.json') as f:
    results = json.load(f)

print("Two-Stage Pipeline Results")
print("="*60)
print(f"Total predictions: {results['metadata']['total_predictions']}")
print(f"Students: {results['metadata']['num_students']}")
print(f"\nStage 1: {results['metadata']['stage1_model']}")
print(f"Stage 2: {results['metadata']['stage2_model']}")

# Sample prediction
sample = results['predictions'][0]
print("\nSample Prediction:")
print(f"Student: {sample['student_id']}")
print(f"Date: {sample['date']}")
print("\nStage 1 Behavioral Predictions:")
for k, v in sample['stage1_behavioral_predictions'].items():
    uncertainty = sample['stage1_uncertainties'][k]
    print(f"  {k}: {v:.2f} ± {uncertainty:.2f}")
print("\nStage 2 Mental Health Predictions:")
for k, v in sample['stage2_mental_health_predictions'].items():
    print(f"  {k}: {v:.2f}")

## Step 4: Uncertainty Analysis

In [ ]:
# Analyze uncertainty propagation
uncertainties = []
for pred in results['predictions']:
    total_uncertainty = pred['error_propagation']['stage1_total_uncertainty']
    uncertainties.append(total_uncertainty)

print("Stage 1 Uncertainty Statistics")
print("="*60)
print(f"Mean total uncertainty: {np.mean(uncertainties):.3f}")
print(f"Std: {np.std(uncertainties):.3f}")
print(f"Min: {np.min(uncertainties):.3f}")
print(f"Max: {np.max(uncertainties):.3f}")
print("\nInterpretation:")
print("  Higher uncertainty in Stage 1 → Less reliable Stage 2 inputs")
print("  → Larger errors in final mental health predictions")

## Step 5: Export Results

In [ ]:
# Already saved to two_stage_predictions.json
# Also save Stage 1 model
print("✓ Results saved:")
print("  - two_stage_predictions.json")
print("  - stage1_behavioral_forecasting.pt")
print("\nDownload these files and add to your project!")